# NOTAS
- BOW con sklearn: inviable porque me elimina las fechas.
- Analzador con stanza : RuntimeError: CUDA error: no kernel image is available for execution on the device.
- Bag of word de todo saldria un diccionario enorme y con un costo computacional inmenso.
- Al tener más lugares donde tener que analizar el contexto, la red neuronal se vuelve más volatil.
- Por NLP pasa algo similiar a lo que pasa en el punto anterior, al tener más texto, tenemos más taggs y el analisis del contexto se vuelve muy complejo, ademas se suma la terminologia, las formulas medicas y las abreviaciones, las cuales los programas de taggeo para el POS se les hace imposible clasificar.

In [5]:
import os 
import glob
import tensorflow as tf
import nltk
#nltk.download()
import stanza 
import spacy_stanza
import string
#stanza.download('es')
import spacy
#!python3 -m spacy download es_core_news_sm
nlp = spacy.load('es_core_news_sm')
import pdf_to_txt_3
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() 
string.punctuation.replace("$",'').replace("=>",'')

'!"#%&\'()*+,-./:;<?@[\\]^_`{|}~'

In [6]:
# Funcion para quitar los acentos 
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b)
    return s

# funcion para permanecer las mayusculas de las secciones
def agg_mayusculas(s):
    replacements =(
            ('motivo de consulta','MOTIVO DE CONSULTA'),
            ('enfermedad actual','ENFERMEDAD ACTUAL'),
            ('examen físico','EXAMEN FÍSICO'),
            ('análisis','ANÁLISIS'),
            ('plan y manejo','PLAN Y MANEJO'),
            ('diagnóstico','DIAGNÓSTICO'),
            ('ordenes','ORDENES'),
            ('órdenes','ÓRDENES'),
            ('interconsultas','INTERCONSULTAS'),
            ('notas enfermeria','NOTAS ENFERMERIA'),
            ('formula médica','FORMULA MÉDICA'),
            ('formatos','FORMATOS'),
            ('recomendaciones','RECOMENDACIONES'),
            ('nota de ingreso','NOTA DE INGRESO'),
            ('observaciones','OBSERVACIONES')
            )

    for a, b in replacements:
        s = s.replace(a, b) 
    return s

# funcion para pasar de texto a numeros 
def translate(text,word_index):
    ''' Esta función usa el diccionario de la bolsa de palabras
        Para transformar un texto en una lista de numeros
        texto -> numeros
        
        -----------
        Parametros:
        -----------
        - text (string) : el string del texto a traducir
         '''
    text = text.split(" ")
    return [word_index.get(palabra,2) for palabra in text]

# funcion para pasar de numeros a texto
def traductor(list_number,reverse_word_index):
    ''' Esta función usa el diccionario de la bolsa de palabras
        Para transformar un texto en una lista de numeros
        numeros -> texto
        
        -----------
        Parametros:
        -----------
        - text (string) : el string del texto a traducir
         '''
    lista = [reverse_word_index.get(numero,'?') for numero in list_number]
    return ' '.join(lista).replace('<PAD>','')



# funcion para crear la bolsa de palabras
def bag(super_string):
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer() 
    words = super_string.split(" ")
    Bag = cv.fit(words)
    #print(sorted(Bag.vocabulary_.items()))
    # ahroa haremos un diccionario con las palabras
    word_index = {k:(v+len(mayusculas)+3) for k,v in sorted(Bag.vocabulary_.items())}
    # El diccionario lo empezamos en v+3 
    # para que las 4 primeras sean caracteres especiales

    # Los caracteres especiales serán 
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2
    word_index["<UNUSED>"] = 3
    for mayuscula in range(len(mayusculas)):
        word_index[mayusculas[mayuscula]] = mayuscula+3

    reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

    return word_index, reverse_word_index


def def_len(seccion):
    seccion = tf.keras.preprocessing.sequence.pad_sequences(seccion, value= word_index["<PAD>"],padding="post", maxlen= 5000 )
    return seccion
    


# creamos función para remover los signos de puntuacion
def remove_punctuation(text):
    puntc = string.punctuation.replace("$",'').replace("=>",'') + "“"
    text_input = nltk.tokenize.word_tokenize(text)
    text_output = [text for text in text_input if text not in puntc]
    return ' '.join(text_output)






# lista de las mayusculas para agregar en el diccionario de la bolsa de palabras
mayusculas = ['ACTUAL', 'ANÁLISIS', 'CONSULTA', 'DE', 'DIAGNÓSTICO', 'ENFERMEDAD', 'ENFERMERIA', 'EXAMEN', 'FORMATOS', 'FORMULA', 'FÍSICO', 'INGRESO', 'INTERCONSULTAS', 'MANEJO', 'MOTIVO', 'MÉDICA', 'NOTA', 'NOTAS', 'OBSERVACIONES', 'ORDENES', 'PLAN', 'RECOMENDACIONES', 'Y', 'ÓRDENES']



In [7]:
# SuperString para Bag_of_words
super_string = '' # donde agregaremos el texto de todos los pdf
secciones = [] # donde agregaremos todas las secciones 

# lista de las secciones tanto en letras como numericas

# 1
motivo_consulta = []
motivo_consulta_n = []

#2
enfermedad_actual = []
enfermedad_actual_n = []

#3
examen_fisico = []
examen_fisico_n = []

#4
analisis = []
analisis_n = []

#5
plan_manejo = []
plan_manejo_n = []

#6
diagnostico = []
diagnostico_n = []

#7
ordenes = []
ordenes_n = []

#8
interconsultas = []
interconsultas_n = []

#9
notas_enfermeria = []
notas_enfermeria_n = []

#10
formula_medica = []
formula_medica_n = []

#11
formatos = []
formatos_n = []

#12
recomendaciones = []
recomendaciones_n = []

#13
notas_ingreso = []
notas_ingreso_n = []

#14
observaciones = []
observaciones_n = []

#15
otros = []
otros_n = []


In [8]:
# preprocesamiento general
textos = glob.glob('*.txt')
for texto in textos:
    with open(texto,'r') as file:
        paciente = file.read()
        paciente = paciente.lower().replace("\n"," ") # quitamos los "\n"
        paciente = agg_mayusculas(paciente) # agregamos las mayusculas de las secciones
        paciente = normalize(paciente) # quitamos los acentos de la data
        paciente = remove_punctuation(paciente)
        super_string = super_string + '   \n   '+paciente  # agregamos parte al super_string
        partes = paciente.split('$ $') # separamos por secciones 
        for parte in partes:
            secciones.append(parte) # general

In [11]:
word_index , reverse_word_index = bag(super_string) # generamos el dicionario de palabras y el inverso.

In [12]:
secciones_n = [] # creamos una lista de todas las secciones en numero 
for sec in secciones:
    seccion = translate(sec,word_index) # traducimos a numeros
    secciones_n.append(seccion) # agregamosa en la lista

In [13]:
secciones_n = tf.keras.preprocessing.sequence.pad_sequences(secciones_n,
                                                     value= word_index["<PAD>"],
                                                     padding="post",
                                                     maxlen= 5000 )
# definimos un len maximo para la lista de palabras numerica

In [14]:
# Agregamos cada seccion a su lista en palabras y numerica correspondiente
for seccion in secciones:
    if 'MOTIVO DE CONSULTA' in seccion:
        motivo_consulta.append(seccion)
        motivo_consulta_n.append(translate(seccion,word_index))

    elif 'ENFERMEDAD ACTUAL' in seccion:
        enfermedad_actual.append(seccion)
        enfermedad_actual_n.append(translate(seccion,word_index))

    elif 'EXAMEN FÍSICO' in seccion:
        examen_fisico.append(seccion)
        examen_fisico_n.append(translate(seccion,word_index))

    elif 'ANÁLISIS' in seccion:
        analisis.append(seccion)
        analisis_n.append(translate(seccion,word_index))

    elif 'PLAN Y MANEJO' in seccion:
        plan_manejo.append(seccion)
        plan_manejo_n.append(translate(seccion,word_index))

    elif 'DIAGNÓSTICO' in seccion:
        diagnostico.append(seccion)
        diagnostico_n.append(translate(seccion,word_index))

    elif ('ORDENES' or 'ÓRDENES') in seccion:
        ordenes.append(seccion)
        ordenes_n.append(translate(seccion,word_index))

    elif 'INTERCONSULTAS' in seccion:
        interconsultas.append(seccion)
        interconsultas_n.append(translate(seccion,word_index))

    elif 'NOTAS ENFERMERIA' in seccion:
        notas_enfermeria.append(seccion)
        notas_enfermeria_n.append(translate(seccion,word_index))

    elif 'FORMULA MÉDICA' in seccion:
        formula_medica.append(seccion)
        formula_medica_n.append(translate(seccion,word_index))

    elif 'FORMATOS' in seccion:
        formatos.append(seccion)
        formatos_n.append(translate(seccion,word_index))

    elif 'RECOMENDACIONES' in seccion:
        recomendaciones.append(seccion)
        recomendaciones_n.append(translate(seccion,word_index))

    elif 'NOTA DE INGRESO' in seccion:
        notas_ingreso.append(seccion)
        notas_ingreso_n.append(translate(seccion,word_index))

    elif 'OBSERVACIONES' in seccion:
        observaciones.append(seccion)
        observaciones_n.append(translate(seccion,word_index))

    else:
        otros.append(secciones)
        otros_n.append(translate(seccion,word_index))


In [15]:
# definimos un len maximo para la lista de palabras numerica para cada seccion 

#1
motivo_consulta_n = def_len(motivo_consulta_n)

#2
enfermedad_actual_n = def_len(enfermedad_actual_n)

#3
examen_fisico_n = def_len(examen_fisico_n)

#4
analisis_n = def_len(analisis_n)

#5
plan_manejo_n = def_len(plan_manejo_n)

#6
diagnostico_n = def_len(diagnostico_n)

#7
ordenes_n = def_len(ordenes_n)

#8
notas_enfermeria_n = def_len(notas_enfermeria_n)

#9
formula_medica_n = def_len(formula_medica_n)

#10
formatos_n = def_len(formatos_n)

#11
recomendaciones_n = def_len(recomendaciones_n)

#12
notas_ingreso_n = def_len(notas_ingreso_n)

#13
observaciones_n = def_len(observaciones_n)

#14
otros_n = def_len(otros_n)
               

In [16]:
import re
# verificamos que este de forma correcta 
i = 4
print(secciones_n[i][:50])
print(' ')
print(' ')
n = traductor(secciones_n[i][:50], reverse_word_index)
print(n)
print(' ')
print(' ')
print(secciones[i])
print(' ')
text_input = nltk.tokenize.word_tokenize(secciones[i])
print(text_input)
match=re.search(r'(\d+/\d+/\d+)',secciones[i])
match.group(1)  

[  2  26 414 834 720 853 332 429   2 799 975 575 245 982 787   2   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 
 
<UNK> ÓRDENES de procedimientos no quirurgico cantidad descripcion <UNK> pl tin ica alta toxic pendiente <UNK>                                  
 
 
 ÓRDENES de procedimientos no quirurgico cantidad descripcion 1 pl tin ica alta toxic pendiente 
 
['ÓRDENES', 'de', 'procedimientos', 'no', 'quirurgico', 'cantidad', 'descripcion', '1', 'pl', 'tin', 'ica', 'alta', 'toxic', 'pendiente']


AttributeError: 'NoneType' object has no attribute 'group'

In [50]:
doc = nlp(secciones[i])
print(doc.text)
for token in doc:
    print(token.text, token.pos_,)

 PLAN Y MANEJO control en 3 meses lab-psa total-rx de torax y eco abdomen total evolucion realizada por carlos alberto rodriguez grosser-fecha 22/04/21 12:43:13 
  SPACE
PLAN PROPN
Y CCONJ
MANEJO PROPN
control NOUN
en ADP
3 NUM
meses NOUN
lab-psa ADJ
total-rx PROPN
de ADP
torax NOUN
y CCONJ
eco ADJ
abdomen NOUN
total ADJ
evolucion NOUN
realizada ADJ
por ADP
carlos NOUN
alberto ADJ
rodriguez NOUN
grosser-fecha ADJ
22/04/21 NUM
12:43:13 NUM


In [2]:
import stanza 
stanza.download('es')

2021-06-10 13:49:56 INFO: Downloading default packages for language: es (Spanish)...
2021-06-10 13:49:58 INFO: File exists: /home/felo/stanza_resources/es/default.zip.
2021-06-10 13:50:02 INFO: Finished downloading models and saved to /home/felo/stanza_resources.


In [3]:
nlp = stanza.Pipeline('es')

2021-06-10 13:50:32 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-06-10 13:50:32 INFO: Use device: gpu
2021-06-10 13:50:32 INFO: Loading: tokenize


RuntimeError: CUDA error: no kernel image is available for execution on the device

In [18]:
secciones[2]

' ENFERMEDAD ACTUAL pacienbte masculino de 51 años con diagnostico linfoma hodgkin clasico estadio ll ipi 3 slp a 5 años 60 secuela de isquemia a nivel de cerebelo sindrome de klippel feil `` completo 6 ciclos de quimioterapia abwd entre el 07-05-20 y el 26 -11-20 continua con dolor en hombros mejoria de parestesias y movilizacion de dedos de las manos e fisico peso 88 kg vigil buen estado general piel se observa pigmentada en la cara y antebrazos no se palpan adenopatias cardio pulmonar normal abdomen blando no doloroso sin masas ni visceromegalias extremidades sin edemas laboratorios 14-01-21 hb 13 hto 38,9 gb 5830 n 49 l 44 plaq 277.000 ca 1.2 gl 88 tg 248 ac urico 6 ct 254 pet-tc 06-01-21 al comparar con estudio previo del 28-04-20 se indetifica resolucion de adenopatias en cadena laterocervical derecha mejoria significatica de conglomerado plastron adenopatico mediastinal en la actualidad se identifican adenopatias aisladas en diferecntes niveles mediastinales descritos con leve i

In [9]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

JJ	Adjectives

NN	Nouns

RB	Adverbs

PRP	Pronouns

VB	Verbs

In [10]:
words =  word_tokenize(secciones[2])
nltk.pos_tag(words)

[('ENFERMEDAD', 'NNP'),
 ('ACTUAL', 'NNP'),
 ('pacienbte', 'NN'),
 ('masculino', 'NN'),
 ('de', 'IN'),
 ('51', 'CD'),
 ('años', 'JJ'),
 ('con', 'NN'),
 ('diagnostico', 'NN'),
 ('linfoma', 'NN'),
 ('hodgkin', 'NN'),
 ('clasico', 'NN'),
 ('estadio', 'NN'),
 ('ll', 'NN'),
 ('ipi', 'NN'),
 ('3', 'CD'),
 ('slp', 'VBD'),
 ('a', 'DT'),
 ('5', 'CD'),
 ('años', 'NN'),
 ('60', 'CD'),
 ('secuela', 'NN'),
 ('de', 'FW'),
 ('isquemia', 'FW'),
 ('a', 'DT'),
 ('nivel', 'NN'),
 ('de', 'IN'),
 ('cerebelo', 'FW'),
 ('sindrome', 'FW'),
 ('de', 'FW'),
 ('klippel', 'FW'),
 ('feil', 'FW'),
 ('``', '``'),
 ('completo', 'NN'),
 ('6', 'CD'),
 ('ciclos', 'NN'),
 ('de', 'FW'),
 ('quimioterapia', 'FW'),
 ('abwd', 'FW'),
 ('entre', 'FW'),
 ('el', 'FW'),
 ('07-05-20', 'JJ'),
 ('y', 'NN'),
 ('el', 'VBZ'),
 ('26', 'CD'),
 ('-11-20', 'NN'),
 ('continua', 'NN'),
 ('con', 'NN'),
 ('dolor', 'NN'),
 ('en', 'IN'),
 ('hombros', 'JJ'),
 ('mejoria', 'FW'),
 ('de', 'FW'),
 ('parestesias', 'FW'),
 ('y', 'FW'),
 ('movilizacion', 

In [25]:
len(super_string)

29688